# Оператор мобильной связи «Мегалайн» 

## Построение модели для задачи классификации, которая выберет подходящий тариф

In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:11,.2f}'.format

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Изучение данных

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv',index_col=0)
df.head()

,minutes,messages,mb_used,is_ultra
calls,,,,
40.00,311.90,83.00,"19,915.42",0
85.00,516.75,56.00,"22,696.96",0
77.00,467.66,86.00,"21,060.45",0
106.00,745.53,81.00,"8,437.39",1
66.00,418.74,1.00,"14,502.75",0


<font color='green'>Подгрузили библиотеки, открыли файл, посмотрели на данные. Пока все просто.

## Деление данных на выборки

Набор данных разбит на 3 группы в соответствии с обычной практикой: Train 60%, Validation 20%, Test 20%.
Поскольку метод train_test_split предназначен для разделения только на 2 группы, мы вызываем его дважды, чтобы разделить на три.

In [3]:
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=12345)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Изучение модели

In [4]:
#for n_estim in range(1,20):
    #model = RandomForestClassifier(random_state=12345, n_estimators=n_estim)
    #model.fit(features_train, target_train)
    #predictions_valid = model.predict(features_valid)
    #print("RandomForest n_estimator =", n_estim, ": ", end='')
    #print(accuracy_score(target_valid, predictions_valid))

In [5]:
for i in range (1, 51, 1) :
    for e in range(1, 20) :
        model = RandomForestClassifier(random_state=12345, n_estimators=i, max_depth=e)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        print("n_estimators=", i, "max_depth=",e, ": ", end='')
        print(accuracy_score(target_valid, predictions_valid))

n_estimators= 1 max_depth= 1 : 0.7293934681181959
n_estimators= 1 max_depth= 2 : 0.7636080870917574
n_estimators= 1 max_depth= 3 : 0.7651632970451011
n_estimators= 1 max_depth= 4 : 0.76049766718507
n_estimators= 1 max_depth= 5 : 0.776049766718507
n_estimators= 1 max_depth= 6 : 0.7542768273716952
n_estimators= 1 max_depth= 7 : 0.7636080870917574
n_estimators= 1 max_depth= 8 : 0.7682737169517885
n_estimators= 1 max_depth= 9 : 0.7636080870917574
n_estimators= 1 max_depth= 10 : 0.7573872472783826
n_estimators= 1 max_depth= 11 : 0.7558320373250389
n_estimators= 1 max_depth= 12 : 0.7200622083981337
n_estimators= 1 max_depth= 13 : 0.7278382581648523
n_estimators= 1 max_depth= 14 : 0.7153965785381027
n_estimators= 1 max_depth= 15 : 0.7371695178849145
n_estimators= 1 max_depth= 16 : 0.702954898911353
n_estimators= 1 max_depth= 17 : 0.6889580093312597
n_estimators= 1 max_depth= 18 : 0.6951788491446346
n_estimators= 1 max_depth= 19 : 0.6702954898911353
n_estimators= 2 max_depth= 1 : 0.75272161741

Random Forest: лучший результат с n_estimators= 18 max_depth= 8 : 0.7978227060653188

## Проверка модели на тестовой выборке

Логистическая регрессия: точность при проверке: 0,674

In [6]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print("Логистическая регрессия:", accuracy_score(target_valid, predictions_valid))

Логистическая регрессия: 0.7076205287713841


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("DecisionTree max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

DecisionTree max_depth = 1 : 0.7542768273716952
DecisionTree max_depth = 2 : 0.7822706065318819
DecisionTree max_depth = 3 : 0.7853810264385692
DecisionTree max_depth = 4 : 0.7884914463452566
DecisionTree max_depth = 5 : 0.7884914463452566


DecisionTree: лучший результат с max_depth = 4 и 5 (0.788)

## Проверка модели на адекватность

In [12]:
#model = RandomForestClassifier(random_state=12345, n_estimators=18)
#model.fit(features_train, target_train)
#predictions_valid = model.predict(features_valid)
#print(accuracy_score(target_valid, predictions_valid))

In [11]:
model = RandomForestClassifier(random_state=12345, n_estimators=18, max_depth= 8)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7978227060653188


Мы сделали сравнение 3 моделей: Random Forest, Logistic Regression и Decision Tree. По результатам, Random Forest (с точностью _estimator = 18) обеспечивает наилучшую точность: 79,7%

In [13]:
predictions_test = model.predict(features_test)
print(accuracy_score(target_test, predictions_test))

0.80248833592535


Последний шаг, это проверка точности на тестовом наборе. Точность остается 80% - что выше целевого показателя 75%, который мы установили

In [10]:
joblib.dump(model, 'PJmodel.joblib')

['PJmodel.joblib']